# データセットから埋め込みを取得

このノートブックでは、大規模なデータセットから埋め込み（ベクトル表現）を取得する方法の例を示します。

## 1. データセットを読み込む

この例で使用するデータセットは、Amazonの「fine-food reviews」です。このデータセットには、2012年10月までにAmazonユーザーが残した合計568,454件の食品レビューが含まれています。我々は、説明のためにこのデータセットの中から最も新しい1,000件のレビューのサブセットを使用します。これらのレビューは英語で書かれており、一般的には肯定的または否定的なものです。各レビューにはProductId、UserId、Score、レビュータイトル（Summary）、およびレビューボディ（Text）が含まれています。

我々はレビューサマリーとレビューテキストを1つの結合されたテキストに結合し、モデルはこの結合されたテキストをエンコードし、1つのベクトル埋め込みを出力します。

このノートブックを実行するには、以下のパッケージをインストールする必要があります：pandas、openai、transformers、plotly、matplotlib、scikit-learn、torch（transformer dep）、torchvision、およびscipy。

In [13]:
# imports
import pandas as pd
import tiktoken

from openai.embeddings_utils import get_embedding


In [14]:
# 埋め込みモデルのパラメータ
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # これはtext-embedding-ada-002のエンコーディングです
max_tokens = 8000  # text-embedding-ada-002の最大トークン数は8191です


In [15]:
# データセットの読み込みと確認
input_datapath = "data/fine_food_reviews_1k.csv"  # スペースを節約するため、事前にフィルタリングされたデータセットを提供します
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Time", "ProductId", "UserId", "Score", "Summary", "Text"]]
df = df.dropna()
df["combined"] = (
    "Title: " + df.Summary.str.strip() + "; Content: " + df.Text.str.strip()
)
df.head(2)


,Time,ProductId,UserId,Score,Summary,Text,combined
0,1351123200,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,1351123200,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, mos...",Title: Arrived in pieces; Content: Not pleased...


In [17]:
# 最新のレビューを1,000件にサブサンプルし、長すぎるサンプルを削除します。
top_n = 1000
df = df.sort_values("Time").tail(top_n * 2)  # 最初に2,000エントリーまで切り取り、半分未満がフィルタリングされると仮定しています
df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# 埋め込むのに長すぎるレビューを省略します
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens].tail(top_n)
len(df)


1000

## 2. 埋め込みを取得し、将来の再利用のために保存する

In [18]:
# READMEに従って環境にAPIキーを設定していることを確認してください：https://github.com/openai/openai-python#usage

# これには数分かかる場合があります
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))
df.to_csv("data/fine_food_reviews_with_embeddings_1k.csv")
